In [4]:
import os
import yaml
from process_data import Task_data
import numpy as np
from transformer_model import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Cuda available: ", torch.cuda.is_available())
print("Current device: ", torch.cuda.current_device())

# Load data
task_config_dir = '../Process_data/postprocessed/2022-10-06'
with open(os.path.join(task_config_dir, 'task_config.yaml')) as file:
    config = yaml.load(file, Loader=yaml.FullLoader)
project_dir = config["project_path"] # Modify this to your need
base_dir = os.path.join(project_dir, config["postprocessed_dir"])
triangulation = 'dlc3d'
template_dir = os.path.join(project_dir, config["postprocessed_dir"],f'transformations/{triangulation}')
individuals = config["individuals"] # The objects that we will place a reference frame on
objs = config["objects"]
data = Task_data(base_dir, triangulation, individuals)
data.dims = ['x', 'y', 'z']

Cuda available:  False


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
model = TrajectoryModel().to(device)

In [ ]:
def regularizer_loss(mat):
    dim = mat.shape[-1]
    ident = torch.eye(dim).to(device)
    reg_loss = torch.mean(torch.norm(torch.bmm(mat, mat.transpose(2, 1)) - ident, dim=(1, 2)))
    return reg_loss

def train_set_transformer(model, t_dataloader, v_dataloader):
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    running_loss = 0
    total_steps = 0
    training_result, training_truth = [], []
    validation_result, validation_truth = [], []
    model.train()
    for data in t_dataloader:
        labels = data.y
        cloud_list = data.pos.split(1000)
        new_clouds = []
        for j in range(len(cloud_list)):
            new_points = cloud_list[j]
            new_points = (new_points - torch.mean(new_points))/torch.std(new_points)
            new_clouds.append(new_points.unsqueeze(0))
        points = torch.cat(new_clouds)

#         points = shift_point_cloud(pclouds)
        points = torch.tensor(points).permute([1, 0, 2])
        optimizer.zero_grad()
        # Pointnet output
#         output, feat_mat = model(points.to(device))
#         reg_loss = regularizer_loss(feat_mat)
#         print(F.nll_loss(output, labels.to(device)), reg_loss*0.001)
#         loss = F.cross_entropy(output, labels.to(device)) + reg_loss*0.001

        # Set Transformer output
        output = model(points.to(device)).squeeze(0)
        loss = F.cross_entropy(output, labels.to(device))
        training_result += [i.item() for i in output.argmax(1)]
        training_truth += [i.item() for i in labels]
        #This is where the model learns by backpropagating
        loss.backward()

        #And optimizes its weights here
        optimizer.step()

        running_loss += loss.item()
        total_steps += 1
    model.eval()
    for data in v_dataloader:
        labels = data.y
        cloud_list = data.pos.split(1000)
        new_clouds = []
        for j in range(len(cloud_list)):
            new_points = cloud_list[j]
            new_points = (new_points - torch.mean(new_points))/torch.std(new_points)
            new_clouds.append(new_points.unsqueeze(0))
        pclouds = torch.cat(new_clouds)
        points = torch.tensor(pclouds).permute([1, 0, 2])
        # Pointnet output
#         output, _ = model(points.to(device))
        # Set Transformer output
        output = model(points.to(device)).squeeze(0)
        validation_result += [i.item() for i in output.argmax(1)]
        validation_truth += [i.item() for i in labels]
    return running_loss/total_steps, training_result, training_truth, validation_result, validation_truth